In [1]:
from _preprocessing import *

params = dict()

# ------------------ configuration ------------------

params['source']                 = 'STEAD'
params['mode']                   = 'generator'
params['window_count']           = 6
params['condition']              = 'M>3'
params['imbalance']              = False
params['config_name']            = make_config(params)
params['file_name']              = params['source'] + '_' + params['config_name'] + '_dump.pkl'
params['model_name']             = 'LSTM_' + params['config_name']
params['output']                 = 'lstm'

# ------------------ preprocessing ------------------

params['output_name']            = 'test_trainer'
params['input_dimention']        = (6000, 3)
params['shuffle']                = True
params['label_type']             = 'gaussian'
params['normalization_mode']     = 'std'
params['augmentation']           = True
params['add_event_r']            = 0.6
params['add_gap_r']              = 0.2
params['add_noise_r']            = 0.5
params['drop_channel_r']         = 0.5
params['shift_event_r']          = 0.9
params['scale_amplitude_r']      = None
params['pre_emphasis']           = False

params['batch_size']             = 36
params['total_data_size']        = 20000
params['T/F_ratio']              = [0.5, 0.5]
params['train_valid_test_split'] = [0.60, 0.20, 0.20]

# ----------------- model-parameter -----------------

params['kernel_size']     = 20
params['embed_size']      = 512
params['hidden_size']     = 512

# ----------------- train-parameter -----------------

params['epochs']                 = 100
params['lr']                     = 0.001

import pprint
pprint.pprint(params)

preprocessing_generator(params)

Using TensorFlow backend.


{'T/F_ratio': [0.5, 0.5],
 'add_event_r': 0.6,
 'add_gap_r': 0.2,
 'add_noise_r': 0.5,
 'augmentation': True,
 'batch_size': 36,
 'condition': 'M>3',
 'config_name': 'W6_M>3_balance',
 'drop_channel_r': 0.5,
 'embed_size': 512,
 'epochs': 100,
 'file_name': 'STEAD_W6_M>3_balance_dump.pkl',
 'hidden_size': 512,
 'imbalance': False,
 'input_dimention': (6000, 3),
 'kernel_size': 20,
 'label_type': 'gaussian',
 'lr': 0.001,
 'mode': 'generator',
 'model_name': 'LSTM_W6_M>3_balance',
 'normalization_mode': 'std',
 'output': 'lstm',
 'output_name': 'test_trainer',
 'pre_emphasis': False,
 'scale_amplitude_r': None,
 'shift_event_r': 0.9,
 'shuffle': True,
 'source': 'STEAD',
 'total_data_size': 20000,
 'train_valid_test_split': [0.6, 0.2, 0.2],
 'window_count': 6}
file exist!!!


In [2]:
from _utils import *

dataset = load_dataset(params)

DEVICE 		:  cuda:1
pid 		:  45548

---------------------------------Data---------------------------------

file name 	: STEAD_W6_M>3_balance_dump.pkl
data length 	: 20000
T/F count 	: 10000 / 10000
T/F ratio 	: 50.0 %
Train size 	: 12000
Valid size 	: 4000
Test size 	: 4000

------------------------------Data Loader------------------------------

X_train shape:  torch.Size([500, 3, 6000])  	type :  torch.FloatTensor
y_train shape:  torch.Size([500, 1, 6])  	type :  torch.FloatTensor


In [ ]:
from _LSTM_Classification import *

train(dataset, params)

In [ ]:
test_inps, test_outps, test_labels = test(dataset)

In [ ]:
outps = np.zeros((500, 6000))
labels = np.zeros((500, 6000))

for jdx in range(500):
    for idx in range(params['window_count']):
        start = idx*dataset['window_size']
        end = (idx+1)*dataset['window_size']
        
        outps[jdx, start:end] = torch.round(test_outps[jdx, 0, idx])*torch.max(test_inps[jdx, 0])*0.8
        labels[jdx, start:end] = test_labels[jdx, 0, idx]*torch.max(test_inps[jdx, 0])

In [ ]:
for idx in range(30):
    plt.figure(figsize=(20,3))
    plt.plot(labels[idx], color='orange' if 1 in test_labels[idx, 0] else 'green')
    plt.plot(test_inps[idx, 0], color='black')
    plt.plot(outps[idx], color='red' if 1 in test_labels[idx, 0] else 'blue')
    plt.show()

###### 